In [ ]:
# Build a movie recommendation engine using python
# Code from: https://www.youtube.com/watch?v=ueKXSupHz6Q&t=679s

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Load the data
from google.colab import files
uploaded = files.upload()


In [ ]:
df_movies = pd.read_csv('IMDb movies.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_movies

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,Le lion,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,"Alexandre Coquelle, Matthieu Le Naour",Monkey Pack Films,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,$ 3507171,NaN,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,"Radek Bajgar, Herman Finkers",Johan Nijenhuis & Co,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,$ 7299062,NaN,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,"Vineesh Aaradya, Vineesh Aaradya",RMCC Productions,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,2019-03-15,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,"Ahmet Faik Akinci, Kasim Uçkan",Gizem Ajans,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",NaN,6.4,194,NaN,NaN,$ 2833,NaN,NaN,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Get count of number of movies in dataset and number of columns
df_movies.shape

(85855, 22)

In [ ]:
# check for any missing values in the important columns
columns = ['title', 'genre', 'duration', 'director', 'actors', 'description']
df_movies[columns].isnull().values.any()

True

In [ ]:
# cleans data of null values
df_mov_clean = df_movies[columns].dropna(0)

In [ ]:
df_mov_clean.isnull().values.any()

False

In [ ]:
# Convert reviews_from_critics to str
#df_mov_clean['reviews_from_critics'] = df_mov_clean['reviews_from_critics'].to_string()
df_mov_clean['duration'] = df_mov_clean['duration'].to_string()

In [ ]:
df_mov_clean.head(4)

,title,genre,duration,director,actors,description
0,Miss Jerry,Romance,0 45\n1 70\n2 53\n3 ...,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.
1,The Story of the Kelly Gang,"Biography, Crime, Drama",0 45\n1 70\n2 53\n3 ...,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...
2,Den sorte drøm,Drama,0 45\n1 70\n2 53\n3 ...,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...
3,Cleopatra,"Drama, History",0 45\n1 70\n2 53\n3 ...,Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...


In [ ]:
df_mov_clean.dtypes

title          object
genre          object
duration       object
director       object
actors         object
description    object
dtype: object

In [ ]:
df_mov_clean = df_mov_clean.reset_index(drop=True)

In [ ]:
df_mov_clean

,title,genre,duration,director,actors,description
0,Miss Jerry,Romance,0 45\n1 70\n2 53\n3 ...,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.
1,The Story of the Kelly Gang,"Biography, Crime, Drama",0 45\n1 70\n2 53\n3 ...,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...
2,Den sorte drøm,Drama,0 45\n1 70\n2 53\n3 ...,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...
3,Cleopatra,"Drama, History",0 45\n1 70\n2 53\n3 ...,Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...
4,L'Inferno,"Adventure, Drama, Fantasy",0 45\n1 70\n2 53\n3 ...,"Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...
...,...,...,...,...,...,...
83601,Pengalila,Drama,0 45\n1 70\n2 53\n3 ...,T.V. Chandran,"Lal, Akshara Kishor, Iniya, Narain, Renji Pani...",An unusual bond between a sixty year old Dalit...
83602,Manoharam,"Comedy, Drama",0 45\n1 70\n2 53\n3 ...,Anvar Sadik,"Vineeth Sreenivasan, Aparna Das, Basil Joseph,...",Manoharan is a poster artist struggling to fin...
83603,Le lion,Comedy,0 45\n1 70\n2 53\n3 ...,Ludovic Colbeau-Justin,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...
83604,De Beentjes van Sint-Hildegard,"Comedy, Drama",0 45\n1 70\n2 53\n3 ...,Johan Nijenhuis,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...


In [ ]:
df_mov_clean.iloc[105]

title                                          Straight Shooting
genre                                                    Western
duration       0         45\n1         70\n2         53\n3   ...
director                                               John Ford
actors         Harry Carey, Duke R. Lee, George Berrell, Moll...
description    Cattleman Flint cuts off farmer Sims' water su...
Name: 105, dtype: object

In [ ]:
df_move_clean_2 = df_mov_clean.drop(104)

In [ ]:
important_features = []

In [ ]:
# create a function to combine the values of the important columns into a single string
def get_important_features(data):
  for i in range(0, data.shape[0]):
    important_features = data['title'][i] + ' ' + data['genre'][i] + ' ' + data['actors'][i] + ' ' + data['description'][i]
    if i==8001:
      break
    else:
      continue
  return important_features

In [ ]:
# create a column to hold the combined strings
get_important_features(df_mov_clean)

'Nana Drama, Romance Charles Boyer, Martine Carol, Walter Chiari, Paul Frankeur, Elisa Cegani, Jean Debucourt, Marguerite Pierry, Dario Michaelis, Dora Doll, Palau, Luisella Boni, Jacqueline Plessis, Germaine Kerjean, Nerio Bernardi, Jacques Tarride The amorous career of a courtesan and how she met her death at the hands of a rejected lover.'

In [ ]:
len(important_features)

8001

In [ ]:
df_mov_clean_1 = df_mov_clean.loc[:8000, ]

In [ ]:
important_features = pd.Series(important_features)

In [ ]:
important_features.values

array(['Miss Jerry Romance Blanche Bayliss, William Courtenay, Chauncey Depew The adventures of a female reporter in the 1890s.',
       'The Story of the Kelly Gang Biography, Crime, Drama Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson True story of notorious Australian outlaw Ned Kelly (1855-80).',
       "Den sorte drøm Drama Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen Two men of high rank are both wooing the beautiful and famous equestrian acrobat Stella. While Stella ignores the jeweler Hirsch, she accepts Count von Waldberg's offer to follow her home, ...",
       ...,
       "L'amore di una grande regina Comedy, History, Romance Romy Schneider, Adrian Hoven, Magda Schneider, Karl Ludwig Diehl, Christl Mardayn, Paul Hörbiger, Rudolf Vogel, Fred Liewehr, Alfred Neugebauer, Otto Treßler, Stefan Skodler, Peter Weck, 

In [ ]:
df_mov_clean_1['important_features'] = important_features

In [ ]:
df_mov_clean_1

,title,genre,duration,director,actors,description,important_features
0,Miss Jerry,Romance,0 45\n1 70\n2 53\n3 ...,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,"Miss Jerry Romance Blanche Bayliss, William Co..."
1,The Story of the Kelly Gang,"Biography, Crime, Drama",0 45\n1 70\n2 53\n3 ...,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,"The Story of the Kelly Gang Biography, Crime, ..."
2,Den sorte drøm,Drama,0 45\n1 70\n2 53\n3 ...,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,"Den sorte drøm Drama Asta Nielsen, Valdemar Ps..."
3,Cleopatra,"Drama, History",0 45\n1 70\n2 53\n3 ...,Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,"Cleopatra Drama, History Helen Gardner, Pearl ..."
4,L'Inferno,"Adventure, Drama, Fantasy",0 45\n1 70\n2 53\n3 ...,"Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,"L'Inferno Adventure, Drama, Fantasy Salvatore ..."
...,...,...,...,...,...,...,...
7996,Monster from the Ocean Floor,"Horror, Sci-Fi",0 45\n1 70\n2 53\n3 ...,Wyott Ordung,"Anne Kimbell, Stuart Wade, Dick Pinner, Wyott ...","Julie, an American on vacation in Mexico, spot...","Monster from the Ocean Floor Horror, Sci-Fi An..."
7997,Il montone a cinque zampe,Comedy,0 45\n1 70\n2 53\n3 ...,Henri Verneuil,"Fernandel, Françoise Arnoul, Andrex, Michel Ar...",A small town seeking publicity tries to bring ...,"Il montone a cinque zampe Comedy Fernandel, Fr..."
7998,L'amore di una grande regina,"Comedy, History, Romance",0 45\n1 70\n2 53\n3 ...,Ernst Marischka,"Romy Schneider, Adrian Hoven, Magda Schneider,...","England, the 19th century. Young Victoria is c...","L'amore di una grande regina Comedy, History, ..."
7999,Anatomia di un delitto,"Crime, Drama, Film-Noir",0 45\n1 70\n2 53\n3 ...,Jerry Hopper,"Sterling Hayden, Gloria Grahame, Gene Barry, M...",A chief of police detectives fired for brutali...,"Anatomia di un delitto Crime, Drama, Film-Noir..."


In [ ]:
df_mov_clean_1.dtypes

title                 object
genre                 object
duration              object
director              object
actors                object
description           object
important_features    object
dtype: object

In [ ]:
# Convert text to matrix of token counts
cm = CountVectorizer().fit_transform(df_mov_clean_1['important_features'].values.astype('U'))

In [ ]:
#Get cosine similarity matrix from count matrix
cs = cosine_similarity(cm)
#print cosine similarity matrix
#shows how similar a movie is to every other movie based on the traits selected
print(cs)

[[1.         0.17920431 0.16062314 ... 0.20751434 0.08536656 0.13994229]
 [0.17920431 1.         0.12761066 ... 0.10467582 0.12918358 0.13897536]
 [0.16062314 0.12761066 1.         ... 0.21110017 0.08684168 0.08541628]
 ...
 [0.20751434 0.10467582 0.21110017 ... 1.         0.09972768 0.07663341]
 [0.08536656 0.12918358 0.08684168 ... 0.09972768 1.         0.07566048]
 [0.13994229 0.13897536 0.08541628 ... 0.07663341 0.07566048 1.        ]]


In [ ]:
# Get shape of cosine similarity matrix
cs.shape

(8001, 8001)

In [ ]:
# Create column of index values
df_mov_clean_1['Movie_id'] = range(len(important_features))

In [ ]:
df_mov_clean_1

,title,genre,duration,director,actors,description,important_features,Movie_id
0,Miss Jerry,Romance,0 45\n1 70\n2 53\n3 ...,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,"Miss Jerry Romance Blanche Bayliss, William Co...",0
1,The Story of the Kelly Gang,"Biography, Crime, Drama",0 45\n1 70\n2 53\n3 ...,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,"The Story of the Kelly Gang Biography, Crime, ...",1
2,Den sorte drøm,Drama,0 45\n1 70\n2 53\n3 ...,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,"Den sorte drøm Drama Asta Nielsen, Valdemar Ps...",2
3,Cleopatra,"Drama, History",0 45\n1 70\n2 53\n3 ...,Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,"Cleopatra Drama, History Helen Gardner, Pearl ...",3
4,L'Inferno,"Adventure, Drama, Fantasy",0 45\n1 70\n2 53\n3 ...,"Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,"L'Inferno Adventure, Drama, Fantasy Salvatore ...",4
...,...,...,...,...,...,...,...,...
7996,Monster from the Ocean Floor,"Horror, Sci-Fi",0 45\n1 70\n2 53\n3 ...,Wyott Ordung,"Anne Kimbell, Stuart Wade, Dick Pinner, Wyott ...","Julie, an American on vacation in Mexico, spot...","Monster from the Ocean Floor Horror, Sci-Fi An...",7996
7997,Il montone a cinque zampe,Comedy,0 45\n1 70\n2 53\n3 ...,Henri Verneuil,"Fernandel, Françoise Arnoul, Andrex, Michel Ar...",A small town seeking publicity tries to bring ...,"Il montone a cinque zampe Comedy Fernandel, Fr...",7997
7998,L'amore di una grande regina,"Comedy, History, Romance",0 45\n1 70\n2 53\n3 ...,Ernst Marischka,"Romy Schneider, Adrian Hoven, Magda Schneider,...","England, the 19th century. Young Victoria is c...","L'amore di una grande regina Comedy, History, ...",7998
7999,Anatomia di un delitto,"Crime, Drama, Film-Noir",0 45\n1 70\n2 53\n3 ...,Jerry Hopper,"Sterling Hayden, Gloria Grahame, Gene Barry, M...",A chief of police detectives fired for brutali...,"Anatomia di un delitto Crime, Drama, Film-Noir...",7999


In [ ]:
# Get title of the movie that user likes
title = 'Miss Jerry'

In [ ]:
# Find the movie id to sort movies
movie_id = df_mov_clean_1[df_mov_clean_1.title == title]['Movie_id'].values[0]

In [ ]:
# Create list of enumerations for similarity scores
scores = list(enumerate(cs[movie_id]))

In [ ]:
# Sort the list of similarity scores
sorted_scores = sorted(scores, key = lambda x:x[1], reverse=True)
sorted_scores=sorted_scores[1:]

In [ ]:
# Create loop to print first 8 movies based on similarity scores
j = 0
print('The 8 Most Recommended Titles to ', title, 'are:\n')
for item in sorted_scores:
  movie_title = df_mov_clean_1[df_mov_clean_1.Movie_id == item[0]]['title'].values[0]
  print(j+1, movie_title)
  j=j+1
  if j>8:
    break

The 8 Most Recommended Titles to  Miss Jerry are:

1 Down to the Sea in Ships
2 The Nigger
3 The Blot
4 She Had to Say Yes
5 North of the Great Divide
6 The Mark of the Whistler
7 The Red Mill
8 La corazzata Potemkin
9 Duello col pirata nero
